# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
from ddpg import Agent
from buffer import ReplayBuffer
from collections import deque,namedtuple
import torch
import torch.nn.functional as F

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="Tennis_Windows_x86_64/Tennis_Windows_x86_64/Tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode = False)[brain_name]
print(env_info.vector_observations)
# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -6.65278625 -1.5
  -0.          0.          6.83172083  6.         -0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -6.4669857  -1.5
   0.          0.         -6.83172083  6.          0.          0.        ]]
Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        print(actions)
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        print(next_states)
        rewards = env_info.rewards                         # get reward (for each agent)
        print(rewards)
        dones = env_info.local_done                        # see if episode finished
        print(dones)
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))                                        
    

[[-0.97236909  0.14139623]
 [ 0.28656044  1.        ]]
[[  0.           0.           0.           0.           0.
    0.           0.           0.          -7.38993645  -1.5
   -0.           0.           6.83172083   5.99607611  -0.
    0.         -10.30704498  -1.55886006 -29.17107201  -0.98100001
    6.83172083   5.91759634 -29.17107201  -0.98100001]
 [  0.           0.           0.           0.           0.
    0.           0.           0.          -6.70024681  -1.5
    0.           0.          -6.83172083   5.99607611   0.
    0.          -5.84056664  -0.98316395   8.5968132    6.21520042
   -6.83172083   5.91759634   8.5968132    6.21520042]]
[0.0, 0.0]
[False, False]
[[-1.         -0.6056547 ]
 [-1.         -0.13071945]]
[[ -7.38993645  -1.5         -0.           0.           6.83172083
    5.99607611  -0.           0.         -10.30704498  -1.55886006
  -29.17107201  -0.98100001   6.83172083   5.91759634 -29.17107201
   -0.98100001 -10.89980125  -1.71581995  -0.          -1.9620

[[ -0.39999998   0.15311816  30.          -3.59479666   4.56356001
    2.23539615  30.          -3.59479666  -2.47042036  -0.26522148
  -14.70420074  -4.57579613   3.66356039   2.86071587 -14.70420074
   -4.57579613  -0.39999998  -0.78166115  30.          -5.5567956
    2.76356006   3.38793588  30.          -5.5567956 ]
 [ -2.65625167  -1.63180363  30.          -9.41497612  -4.56356001
    2.23539615  30.          -9.41497612  -0.39999998  -0.99066353
   30.           6.01900053  -3.66356039   2.86071587  30.
    6.01900053  -1.70330143  -0.44762343  -7.03301764   5.03800106
   -2.76356006   3.38793588  -7.03301764   5.03800106]]
[0.0, 0.0]
[False, False]
[[-1.          0.58152072]
 [-0.55163846  0.95951727]]
[[-2.47042036e+00 -2.65221477e-01 -1.47042007e+01 -4.57579613e+00
   3.66356039e+00  2.86071587e+00 -1.47042007e+01 -4.57579613e+00
  -3.99999976e-01 -7.81661153e-01  3.00000000e+01 -5.55679560e+00
   2.76356006e+00  3.38793588e+00  3.00000000e+01 -5.55679560e+00
  -3.99999976e+00

[[ -3.07149291   0.8110469    5.47526026  -0.45559746  -6.23643827
    3.26463795   5.47526026  -0.45559746  -0.75968975   0.70662725
   23.11803055  -1.43659747  -7.13643742   2.7127583   23.11803055
   -1.43659747  -0.52906984   0.50410748  23.54650688  -2.41759729
   -8.03643703   2.06277871  23.54650688  -2.41759729]
 [ -7.13403177  -0.41272104  -3.28100681  -4.77199602   6.23643827
    3.26463795  -3.28100681  -4.77199602  -4.28834152  -0.94878066
   28.45689964  -5.75299549   7.13643742   2.7127583   28.45689964
   -5.75299549  -7.28834105  -1.5829401  -30.          -6.73399496
    8.03643703   2.06277871 -30.          -6.73399496]]
[0.0, 0.0]
[False, False]
[[-0.95007808 -1.        ]
 [ 0.8228284  -0.32387191]]
[[ -0.75968975   0.70662725  23.11803055  -1.43659747  -7.13643742
    2.7127583   23.11803055  -1.43659747  -0.52906984   0.50410748
   23.54650688  -2.41759729  -8.03643703   2.06277871  23.54650688
   -2.41759729  -3.85023427   0.20348775 -28.50234222  -3.39859676
   -

[[-1.14365587e+01 -6.01273894e-01 -3.00000000e+01  5.43040085e+00
   6.69487906e+00  3.66522074e+00 -3.00000000e+01  5.43040085e+00
  -8.43655682e+00 -1.17093801e-01  3.00000000e+01  4.44940138e+00
   6.69487906e+00  2.93928099e+00  3.00000000e+01  4.44940138e+00
  -9.41731071e+00  2.68986344e-01 -9.80755615e+00  3.46840191e+00
   6.69487906e+00  2.11524105e+00 -9.80755615e+00  3.46840191e+00]
 [-1.08997889e+01  8.49136829e-01 -9.53674316e-06  1.31020260e+00
  -6.69487906e+00  3.66522074e+00 -9.53674316e-06  1.31020260e+00
  -7.89978647e+00  9.21297073e-01  3.00000000e+01  3.29202533e-01
  -6.69487906e+00  2.93928099e+00  3.00000000e+01  3.29202533e-01
  -4.89978695e+00  8.95357311e-01  3.00000000e+01 -6.51797473e-01
  -6.69487906e+00  2.11524105e+00  3.00000000e+01 -6.51797473e-01]]
[0.0, 0.0]
[False, False]
[[-0.2839306   0.9267418 ]
 [ 0.12234868 -0.4300907 ]]
[[ -8.43655682  -0.1170938   30.           4.44940138   6.69487906
    2.93928099  30.           4.44940138  -9.41731071   0

[[ -9.88246155  -1.18375123  -4.36500931   6.41140032   6.55063343
    4.91697645  -4.36500931   6.41140032  -7.5546093   -0.60147107
   23.27850723   5.43040085   6.55063343   4.40685654  23.27850723
    5.43040085  -4.55460978  -0.11729097  30.           4.44940138
    6.55063343   3.79863667  30.           4.44940138]
 [ -0.52161247  -0.49632382  23.91937447   5.23420095  -6.55063343
    4.91697645  23.91937447   5.23420095  -0.54746658  -0.0317637
   22.62667084   4.25320148  -6.55063343   4.40685654  22.62667084
    4.25320148  -3.53471255   0.33469647 -25.3471241    3.27220201
   -6.55063343   3.79863667 -25.3471241    3.27220201]]
[0.0, 0.0]
[False, False]
[[-0.62636707 -0.32688791]
 [-0.75360921 -1.        ]]
[[ -7.5546093   -0.60147107  23.27850723   5.43040085   6.55063343
    4.40685654  23.27850723   5.43040085  -4.55460978  -0.11729097
   30.           4.44940138   6.55063343   3.79863667  30.
    4.44940138  -6.43371058   0.26878917 -18.79101181   3.46840191
    6.5506334

[[ -7.18080616  -1.26590788  12.48409843  -6.34159517  -1.2347182
    6.15457296  12.48409843  -6.34159517  -4.87052774  -1.95892727
   23.10279465  -7.32259464  -2.13471794   6.19129324  23.10279465
   -7.32259464  -7.87052774  -1.28247142 -30.           6.60760021
   -3.03471828   6.12991333 -30.           6.60760021]
 [-10.82208157   0.64615881  -1.54203951   2.29120255   1.2347182
    6.15457296  -1.54203951   2.29120255  -7.82207918   0.81641901
   30.           1.3102026    2.13471794   6.19129324  30.
    1.3102026  -10.82208157   0.88857925 -30.           0.32920253
    3.03471828   6.12991333 -30.           0.32920253]]
[0.0, 0.0]
[False, False]
[[-0.96955851 -0.07895841]
 [ 0.64426714 -0.27175521]]
[[ -4.87052774  -1.95892727  23.10279465  -7.32259464  -2.13471794
    6.19129324  23.10279465  -7.32259464  -7.87052774  -1.28247142
  -30.           6.60760021  -3.03471828   6.12991333 -30.
    6.60760021 -10.77920151  -0.6805712  -29.08675575   5.62660074
   -3.93471861   5.970

[[ -6.61764145  -1.5         -0.           0.           7.76153517
    5.99607611  -0.           0.          -9.52958107  -0.98316395
  -29.11938286   6.21520042   7.76153517   5.91759634 -29.11938286
    6.21520042 -10.9169817   -0.42050385  -0.           5.23420095
    7.76153517   5.74101639  -0.           5.23420095]
 [ -6.997437    -1.5          0.           0.          -7.76153517
    5.99607611   0.           0.          -6.65565205  -1.55886006
    3.41784763  -0.98100001  -7.76153517   5.91759634   3.41784763
   -0.98100001  -3.65565276  -1.71581995  30.          -1.96200001
   -7.76153517   5.74101639  30.          -1.96200001]]
[0.0, 0.0]
[False, False]
[[ 0.90807253 -0.18615692]
 [-0.10953457  0.37033843]]
[[ -9.52958107  -0.98316395 -29.11938286   6.21520042   7.76153517
    5.91759634 -29.11938286   6.21520042 -10.9169817   -0.42050385
   -0.           5.23420095   7.76153517   5.74101639  -0.
    5.23420095  -8.19276333   0.0440563   27.24217606   4.25320148
    7.761535

[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -6.37251186 -1.5
  -0.          0.          6.04292488  5.96076012 -0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -6.46944427 -1.5
   0.          0.         -6.04292488  5.96076012  0.          0.        ]]
[0.0, -0.009999999776482582]
[True, True]
Score (max over agents) from episode 4: 0.0
[[ 1.         -1.        ]
 [-0.05953783 -0.06604741]]
[[ 0.          0.          0.          0.          0.          0.
   0.          0.         -6.66271734 -1.5        -0.          0.
   6.04292488  5.94114017 -0.          0.         -3.66271806 -1.55886006
  30.         -0.98100001  6.04292488  5.78418016 30.         -0.98100001]
 [ 0.          0.          0.

[[ -8.44760513  -0.1170938    4.71158743   4.44940138   4.79637861
    2.35852146   4.71158743   4.44940138  -7.93538141   0.26898634
    5.12221432   3.46840191   3.89637923   3.10156131   5.12221432
    3.46840191  -4.93538189   0.55696654  30.           2.48740244
    2.9963789    3.74650121  30.           2.48740244]
 [ -6.10100889   0.2816439   -0.99590909  -4.11447573  -4.79637861
    2.35852146  -0.99590909  -4.11447573  -5.44630671  -0.18866366
    6.54701853  -5.0954752   -3.89637923   3.10156131   6.54701853
   -5.0954752   -8.18017673  -0.75707102 -27.3386898   -6.07647467
   -2.9963789    3.74650121 -27.3386898   -6.07647467]]
[0.0, 0.0]
[False, False]
[[-1.         -1.        ]
 [ 0.60244759  1.        ]]
[[ -7.93538141   0.26898634   5.12221432   3.46840191   3.89637923
    3.10156131   5.12221432   3.46840191  -4.93538189   0.55696654
   30.           2.48740244   2.9963789    3.74650121  30.
    2.48740244  -7.93538141   0.74684674 -30.           1.50640261
    2.096378

[[ -4.56034565  -1.13469064  21.2345295    6.41140032  -6.00361967
    4.80040264  21.2345295    6.41140032  -5.64063692  -0.55241048
  -10.80290699   5.43040085  -6.90361881   4.36624289 -10.80290699
    5.43040085  -7.06818962  -0.06823036 -14.27552128   4.44940138
   -7.80361795   3.83398318 -14.27552128   4.44940138]
 [ -5.09768677   0.74684674  -6.64187431   1.50640261   6.00361967
    4.80040264  -6.64187431   1.50640261  -4.61330843   0.83862698
    4.84378099   0.52540255   6.90361881   4.36624289   4.84378099
    0.52540255  -7.61330795   0.83230716 -30.          -0.45559746
    7.80361795   3.83398318 -30.          -0.45559746]]
[0.0, 0.0]
[False, False]
[[ 0.54768319  1.        ]
 [-1.          0.82493487]]
[[ -5.64063692  -0.55241048 -10.80290699   5.43040085  -6.90361881
    4.36624289 -10.80290699   5.43040085  -7.06818962  -0.06823036
  -14.27552128   4.44940138  -7.80361795   3.83398318 -14.27552128
    4.44940138  -5.4251399    0.31784981  16.43049622   3.46840191
   -

When finished, you can close the environment.

In [6]:
def vector_avg(states):
    state_1 = np.array(states[0])
    state_2 = np.array(states[1])
    [state_1_1, state_1_2, state_1_3] = np.split(state_1, 3)
    [state_2_1, state_2_2, state_2_3] = np.split(state_2, 3)
    state_1 = (state_1_1 + state_1_2 + state_1_3)/3
    state_2 = (state_2_1 + state_2_2 + state_2_3)/3
    return [state_1, state_2]

class MADDPG():
    def __init__(self, discount_factor=0.95, tau=0.02):
        super(MADDPG, self).__init__()
        self.maddpg = [Agent(8, 16, 8, 2, 10, 16, 8),
                      Agent(8, 16, 8, 2, 10, 16, 8)]
        self.discount_factor = discount_factor
        self.tau = tau
        self.iter = 0
        self.memory = ReplayBuffer() #defined in the function setup
        self.t_step = 0
    def get_actors(self):
        local_actors = [agent.actor_local for agent in self.maddpg]
        return local_actors
    
    def get_target_actors(self):
        target_actors = [agent.actor_target for agent in self.maddpg]
        return target_actors
    
    def act(self, states, noise=0.0):
        actions = [agent.act(state) for agent, state in zip(self.maddpg, states)]
        return actions
    
    def target_act(self, states, noise=0.0):
        target_actions = [agent.target_act(state) for agent, state in zip(self.maddpg, states)]
        return target_actions
        
    def step(self, state, action, reward, next_state, done, Update_every = 3, Batch = 64):
        self.memory.add(state, action, reward, next_state, done)
        Update_times = 2
        self.t_step = (self.t_step + 1) % Update_every
        if len(self.memory) > Batch and self.t_step == 0:
            for _ in range(Update_times):
                experiences = self.memory.sample()
                [self.learn(experiences, agent, gamma = 0.99) for agent in range(2)]
    def learn(self, experiences, agent_number, gamma):
        states, actions, rewards, next_states, dones = experiences
        agent = self.maddpg[agent_number]
        agent.critic_optimizer.zero_grad()
        actions_next = torch.from_numpy(agent.target_act(next_states[agent_number])).detach().to(device)
        critic_input = torch.cat((next_states[agent_number], actions_next.float()), dim = 0).to(device)
        Q_targets_next = agent.critic_target(critic_input).detach()
        Q_targets = rewards + (gamma*Q_targets_next*(1-dones))
        critic_input = torch.cat((states[agent_number], actions[agent_number]), dim = 0).to(device)
        Q_expected = agent.critic_local(critic_input)
        critic_loss = F.mse_loss(Q_expected, Q_targets)
        agent.critic_optimizer.zero_grad()
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm_(agent.critic_local.parameters(), 1)
        agent.critic_optimizer.step()
        actions_pred = (agent.act(states[agent_number].to('cpu').numpy()))
        actions_pred = torch.from_numpy(actions_pred).to(device)
        
        critic_input = torch.cat((states[agent_number].float(), (actions_pred).float()), dim = 0).to(device)
        actor_loss = -agent.critic_local(critic_input).mean()
        
        agent.actor_optimizer.zero_grad()
        actor_loss.backward()
        agent.actor_optimizer.step()
        
        self.soft_update(agent.critic_local, agent.critic_target)
        self.soft_update(agent.actor_local, agent.actor_target)
    def soft_update(self, local_model, target_model, tau=1e-3):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0 - tau)*target_param.data)
    


state = vector_avg(states)
print(state[0])
print(state[1])


[-2.04676962 -0.5         0.          0.         -2.37247276  1.99607611
  0.          0.        ]
[-2.65247377 -0.5         0.          0.          2.37247276  1.99607611
  0.          0.        ]


In [ ]:
maddpg = MADDPG()

def main():
    print_every = 10
    scores_deque = deque(maxlen = print_every)
    scores = []
    num_episodes = 5000
    episode_length = 80
    batchsize = 64
    noise = 2
    noise_reduction = 0.9999
    episode_per_update = 5
    buffer = ReplayBuffer(int(1e6))
    t_max = 300
    for episode in range(0,num_episodes):
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        score = np.zeros(num_agents)
        for t in range(t_max):
            states = vector_avg(states)
            actions = maddpg.act((states), noise = noise)
            noise *= noise_reduction
            actions = np.clip(actions, -1, 1)
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            score += (env_info.rewards)
            maddpg.step(states, actions, rewards, vector_avg(next_states), dones)
            states = next_states
            if np.any(dones):                                  # exit loop if episode finished
                break
        scores_deque.append(score)
        scores.append(score)        
        print("\r Episode {}\tAverage Score: {:.2f}".format(episode, np.max(scores_deque)), end = "")
        torch.save(maddpg.maddpg[0].actor_local.state_dict(), "checkpoint_actor_agent1.pth")
        torch.save(maddpg.maddpg[0].critic_local.state_dict(), "checkpoint_critic_agent1.pth")
        torch.save(maddpg.maddpg[1].actor_local.state_dict(), "checkpoint_actor_agent2.pth")
        torch.save(maddpg.maddpg[1].critic_local.state_dict(), "checkpoint_critic_agent2.pth")
scores = main()

 Episode 3	Average Score: 0.00

C:\Users\Nikhil Masinete\Anaconda3\envs\drlnd\lib\site-packages\ipykernel_launcher.py:54: UserWarning: Using a target size (torch.Size([64, 2])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


 Episode 737	Average Score: 0.00

In [ ]:
print(actions)
print(states)
critic_input = np.concatenate((states[0], actions[0]), axis = None)
print(critic_input)

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```